## LightGBM + Catboost on tabular data

Tabular data LGBM + Catboost strategy found from here: 
https://www.kaggle.com/code/vyacheslavbolotin/lightgbm-catboost-with-new-features#Competition-Metric

## Imports

In [2]:
import random
import numpy as np
import pandas as pd
import pandas.api.types
import matplotlib.pyplot as plt

from sklearn.preprocessing import OrdinalEncoder
from sklearn.metrics import roc_curve, auc, roc_auc_score
from sklearn.model_selection import GroupKFold, StratifiedGroupKFold
from sklearn.ensemble import VotingClassifier

import catboost as cb
import lightgbm as lgb

## Feature Engineering

In [3]:
df_train = pd.read_csv("../data/train-metadata.csv")
df_test = pd.read_csv("../data/test-metadata.csv")

def feature_engineering(df):
    # New features to try...
    df["lesion_size_ratio"]              = df["tbp_lv_minorAxisMM"] / df["clin_size_long_diam_mm"]
    df["lesion_shape_index"]             = df["tbp_lv_areaMM2"] / (df["tbp_lv_perimeterMM"] ** 2)
    df["hue_contrast"]                   = (df["tbp_lv_H"] - df["tbp_lv_Hext"]).abs()
    df["luminance_contrast"]             = (df["tbp_lv_L"] - df["tbp_lv_Lext"]).abs()
    df["lesion_color_difference"]        = np.sqrt(df["tbp_lv_deltaA"] ** 2 + df["tbp_lv_deltaB"] ** 2 + df["tbp_lv_deltaL"] ** 2)
    df["border_complexity"]              = df["tbp_lv_norm_border"] + df["tbp_lv_symm_2axis"]
    df["color_uniformity"]               = df["tbp_lv_color_std_mean"] / df["tbp_lv_radial_color_std_max"]
    
    df["3d_position_distance"]           = np.sqrt(df["tbp_lv_x"] ** 2 + df["tbp_lv_y"] ** 2 + df["tbp_lv_z"] ** 2) 
    df["perimeter_to_area_ratio"]        = df["tbp_lv_perimeterMM"] / df["tbp_lv_areaMM2"]
    df["area_to_perimeter_ratio"]        = df["tbp_lv_areaMM2"] / df["tbp_lv_perimeterMM"]
    df["lesion_visibility_score"]        = df["tbp_lv_deltaLBnorm"] + df["tbp_lv_norm_color"]
    df["combined_anatomical_site"]       = df["anatom_site_general"] + "_" + df["tbp_lv_location"]
    df["symmetry_border_consistency"]    = df["tbp_lv_symm_2axis"] * df["tbp_lv_norm_border"]
    df["consistency_symmetry_border"]    = df["tbp_lv_symm_2axis"] * df["tbp_lv_norm_border"] / (df["tbp_lv_symm_2axis"] + df["tbp_lv_norm_border"])
    
    df["color_consistency"]              = df["tbp_lv_stdL"] / df["tbp_lv_Lext"]
    df["consistency_color"]              = df["tbp_lv_stdL"] * df["tbp_lv_Lext"] / (df["tbp_lv_stdL"] + df["tbp_lv_Lext"])
    df["size_age_interaction"]           = df["clin_size_long_diam_mm"] * df["age_approx"]
    df["hue_color_std_interaction"]      = df["tbp_lv_H"] * df["tbp_lv_color_std_mean"]
    df["lesion_severity_index"]          = (df["tbp_lv_norm_border"] + df["tbp_lv_norm_color"] + df["tbp_lv_eccentricity"]) / 3
    df["shape_complexity_index"]         = df["border_complexity"] + df["lesion_shape_index"]
    df["color_contrast_index"]           = df["tbp_lv_deltaA"] + df["tbp_lv_deltaB"] + df["tbp_lv_deltaL"] + df["tbp_lv_deltaLBnorm"]
    
    df["log_lesion_area"]                = np.log(df["tbp_lv_areaMM2"] + 1)
    df["normalized_lesion_size"]         = df["clin_size_long_diam_mm"] / df["age_approx"]
    df["mean_hue_difference"]            = (df["tbp_lv_H"] + df["tbp_lv_Hext"]) / 2
    df["std_dev_contrast"]               = np.sqrt((df["tbp_lv_deltaA"] ** 2 + df["tbp_lv_deltaB"] ** 2 + df["tbp_lv_deltaL"] ** 2) / 3)
    df["color_shape_composite_index"]    = (df["tbp_lv_color_std_mean"] + df["tbp_lv_area_perim_ratio"] + df["tbp_lv_symm_2axis"]) / 3
    df["3d_lesion_orientation"]          = np.arctan2(df_train["tbp_lv_y"], df_train["tbp_lv_x"])
    df["overall_color_difference"]       = (df["tbp_lv_deltaA"] + df["tbp_lv_deltaB"] + df["tbp_lv_deltaL"]) / 3
    
    df["symmetry_perimeter_interaction"] = df["tbp_lv_symm_2axis"] * df["tbp_lv_perimeterMM"]
    df["comprehensive_lesion_index"]     = (df["tbp_lv_area_perim_ratio"] + df["tbp_lv_eccentricity"] + df["tbp_lv_norm_color"] + df["tbp_lv_symm_2axis"]) / 4
    df["color_variance_ratio"]           = df["tbp_lv_color_std_mean"] / df["tbp_lv_stdLExt"]
    df["border_color_interaction"]       = df["tbp_lv_norm_border"] * df["tbp_lv_norm_color"]
    df["size_color_contrast_ratio"]      = df["clin_size_long_diam_mm"] / df["tbp_lv_deltaLBnorm"]
    df["age_normalized_nevi_confidence"] = df["tbp_lv_nevi_confidence"] / df["age_approx"]
    df["color_asymmetry_index"]          = df["tbp_lv_radial_color_std_max"] * df["tbp_lv_symm_2axis"]
    
    df["3d_volume_approximation"]        = df["tbp_lv_areaMM2"] * np.sqrt(df["tbp_lv_x"]**2 + df["tbp_lv_y"]**2 + df["tbp_lv_z"]**2)
    df["color_range"]                    = (df["tbp_lv_L"] - df["tbp_lv_Lext"]).abs() + (df["tbp_lv_A"] - df["tbp_lv_Aext"]).abs() + (df["tbp_lv_B"] - df["tbp_lv_Bext"]).abs()
    df["shape_color_consistency"]        = df["tbp_lv_eccentricity"] * df["tbp_lv_color_std_mean"]
    df["border_length_ratio"]            = df["tbp_lv_perimeterMM"] / (2 * np.pi * np.sqrt(df["tbp_lv_areaMM2"] / np.pi))
    df["age_size_symmetry_index"]        = df["age_approx"] * df["clin_size_long_diam_mm"] * df["tbp_lv_symm_2axis"]
    df["index_age_size_symmetry"]        = df["age_approx"] * df["tbp_lv_areaMM2"] * df["tbp_lv_symm_2axis"]
    # Until here..
    # df['np1']                           = np.sqrt(df["tbp_lv_deltaB"]**2 + df["tbp_lv_deltaL"]**2 + df["tbp_lv_deltaLB"]**2) / (df["tbp_lv_deltaB"] + df["tbp_lv_deltaL"] + df["tbp_lv_deltaLB"])
    # df['np2']                           = (df["tbp_lv_deltaA"] + df["tbp_lv_deltaLB"]) / np.sqrt(df["tbp_lv_deltaA"]**2 + df["tbp_lv_deltaLB"]**2)
    # df['np3']                           = ?
    # ...
    # df['npn']                           = ?
    
    new_num_cols = [
        "lesion_size_ratio",             # tbp_lv_minorAxisMM      / clin_size_long_diam_mm
        "lesion_shape_index",            # tbp_lv_areaMM2          / tbp_lv_perimeterMM **2
        "hue_contrast",                  # tbp_lv_H                - tbp_lv_Hext              abs
        "luminance_contrast",            # tbp_lv_L                - tbp_lv_Lext              abs
        "lesion_color_difference",       # tbp_lv_deltaA **2       + tbp_lv_deltaB **2 + tbp_lv_deltaL **2  sqrt  
        "border_complexity",             # tbp_lv_norm_border      + tbp_lv_symm_2axis
        "color_uniformity",              # tbp_lv_color_std_mean   / tbp_lv_radial_color_std_max
        
        "3d_position_distance",          # tbp_lv_x **2 + tbp_lv_y **2 + tbp_lv_z **2  sqrt
        "perimeter_to_area_ratio",       # tbp_lv_perimeterMM      / tbp_lv_areaMM2
        "area_to_perimeter_ratio",       # tbp_lv_areaMM2          / tbp_lv_perimeterMM
        "lesion_visibility_score",       # tbp_lv_deltaLBnorm      + tbp_lv_norm_color
        # "combined_anatomical_site"      # anatom_site_general     + "_" + tbp_lv_location ! categorical feature
        "symmetry_border_consistency",   # tbp_lv_symm_2axis       * tbp_lv_norm_border
        "consistency_symmetry_border",   # tbp_lv_symm_2axis       * tbp_lv_norm_border / (tbp_lv_symm_2axis + tbp_lv_norm_border)
        
        "color_consistency",             # tbp_lv_stdL             / tbp_lv_Lext
        "consistency_color",             # tbp_lv_stdL*tbp_lv_Lext / tbp_lv_stdL + tbp_lv_Lext
        "size_age_interaction",          # clin_size_long_diam_mm  * age_approx
        "hue_color_std_interaction",     # tbp_lv_H                * tbp_lv_color_std_mean
        "lesion_severity_index",         # tbp_lv_norm_border      + tbp_lv_norm_color + tbp_lv_eccentricity / 3
        "shape_complexity_index",        # border_complexity       + lesion_shape_index
        "color_contrast_index",          # tbp_lv_deltaA + tbp_lv_deltaB + tbp_lv_deltaL + tbp_lv_deltaLBnorm
        
        "log_lesion_area",               # tbp_lv_areaMM2          + 1  np.log
        "normalized_lesion_size",        # clin_size_long_diam_mm  / age_approx
        "mean_hue_difference",           # tbp_lv_H                + tbp_lv_Hext    / 2
        "std_dev_contrast",              # tbp_lv_deltaA **2 + tbp_lv_deltaB **2 + tbp_lv_deltaL **2   / 3  np.sqrt
        "color_shape_composite_index",   # tbp_lv_color_std_mean   + bp_lv_area_perim_ratio + tbp_lv_symm_2axis   / 3
        "3d_lesion_orientation",         # tbp_lv_y                , tbp_lv_x  np.arctan2
        "overall_color_difference",      # tbp_lv_deltaA           + tbp_lv_deltaB + tbp_lv_deltaL   / 3
        
        "symmetry_perimeter_interaction",# tbp_lv_symm_2axis       * tbp_lv_perimeterMM
        "comprehensive_lesion_index",    # tbp_lv_area_perim_ratio + tbp_lv_eccentricity + bp_lv_norm_color + tbp_lv_symm_2axis   / 4
        "color_variance_ratio",          # tbp_lv_color_std_mean   / tbp_lv_stdLExt
        "border_color_interaction",      # tbp_lv_norm_border      * tbp_lv_norm_color
        "size_color_contrast_ratio",     # clin_size_long_diam_mm  / tbp_lv_deltaLBnorm
        "age_normalized_nevi_confidence",# tbp_lv_nevi_confidence  / age_approx
        "color_asymmetry_index",         # tbp_lv_symm_2axis       * tbp_lv_radial_color_std_max
        
        "3d_volume_approximation",       # tbp_lv_areaMM2          * sqrt(tbp_lv_x**2 + tbp_lv_y**2 + tbp_lv_z**2)
        "color_range",                   # abs(tbp_lv_L - tbp_lv_Lext) + abs(tbp_lv_A - tbp_lv_Aext) + abs(tbp_lv_B - tbp_lv_Bext)
        "shape_color_consistency",       # tbp_lv_eccentricity     * tbp_lv_color_std_mean
        "border_length_ratio",           # tbp_lv_perimeterMM      / pi * sqrt(tbp_lv_areaMM2 / pi)
        "age_size_symmetry_index",       # age_approx              * clin_size_long_diam_mm * tbp_lv_symm_2axis
         #"index_age_size_symmetry",      # age_approx              * sqrt(tbp_lv_areaMM2 * tbp_lv_symm_2axis)
        "index_age_size_symmetry",       # age_approx              * tbp_lv_areaMM2 * tbp_lv_symm_2axis
         # Until here..
         # 'np1',                         # in case of a positive manifestation
         # 'np2',                         # in case of a positive manifestation
         # 'np3'                          # = ?
         # ...
         # 'npn'                          # = ?
    ]
    
    
    
    
    # The following features have been added:
    
    # - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - -
    
    # "area_to_perimeter_ratio",       # tbp_lv_areaMM2          / tbp_lv_perimeterMM
    # "consistency_symmetry_border",   # tbp_lv_symm_2axis       * tbp_lv_norm_border / (tbp_lv_symm_2axis + tbp_lv_norm_border)
    # "consistency_color",             # tbp_lv_stdL*tbp_lv_Lext / tbp_lv_stdL + tbp_lv_Lext
    # "index_age_size_symmetry",       # age_approx              * tbp_lv_areaMM2 * tbp_lv_symm_2axis
    
    # - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - -
    
    # They were added all at once and by instinct.
    # I think this is not quite right, it would be better to add them one by one
    # and then visually look(even better is a machine gun) at the result
     
    # Working only with the taboo values of expert Andreas Bisi showed that it is possible to work 
    # with a smaller number of functions available to us, 
    # so in the next versions we will try to remove some functions rather than add them.
    
    
    
    
    
    new_cat_cols = ["combined_anatomical_site"]
    
    return df, new_num_cols, new_cat_cols


num_cols = [
    'age_approx',                        # Approximate age of patient at time of imaging.
    'clin_size_long_diam_mm',            # Maximum diameter of the lesion (mm).+
    'tbp_lv_A',                          # A inside  lesion.+
    'tbp_lv_Aext',                       # A outside lesion.+
    'tbp_lv_B',                          # B inside  lesion.+
    'tbp_lv_Bext',                       # B outside lesion.+ 
    'tbp_lv_C',                          # Chroma inside  lesion.+
    'tbp_lv_Cext',                       # Chroma outside lesion.+
    'tbp_lv_H',                          # Hue inside the lesion; calculated as the angle of A* and B* in LAB* color space. Typical values range from 25 (red) to 75 (brown).+
    'tbp_lv_Hext',                       # Hue outside lesion.+
    'tbp_lv_L',                          # L inside lesion.+
    'tbp_lv_Lext',                       # L outside lesion.+
    'tbp_lv_areaMM2',                    # Area of lesion (mm^2).+
    'tbp_lv_area_perim_ratio',           # Border jaggedness, the ratio between lesions perimeter and area. Circular lesions will have low values; irregular shaped lesions will have higher values. Values range 0-10.+
    'tbp_lv_color_std_mean',             # Color irregularity, calculated as the variance of colors within the lesion's boundary.
    'tbp_lv_deltaA',                     # Average A contrast (inside vs. outside lesion).+
    'tbp_lv_deltaB',                     # Average B contrast (inside vs. outside lesion).+
    'tbp_lv_deltaL',                     # Average L contrast (inside vs. outside lesion).+
    'tbp_lv_deltaLB',                    #
    'tbp_lv_deltaLBnorm',                # Contrast between the lesion and its immediate surrounding skin. Low contrast lesions tend to be faintly visible such as freckles; high contrast lesions tend to be those with darker pigment. Calculated as the average delta LB of the lesion relative to its immediate background in LAB* color space. Typical values range from 5.5 to 25.+
    'tbp_lv_eccentricity',               # Eccentricity.+
    'tbp_lv_minorAxisMM',                # Smallest lesion diameter (mm).+
    'tbp_lv_nevi_confidence',            # Nevus confidence score (0-100 scale) is a convolutional neural network classifier estimated probability that the lesion is a nevus. The neural network was trained on approximately 57,000 lesions that were classified and labeled by a dermatologist.+,++
    'tbp_lv_norm_border',                # Border irregularity (0-10 scale); the normalized average of border jaggedness and asymmetry.+
    'tbp_lv_norm_color',                 # Color variation (0-10 scale); the normalized average of color asymmetry and color irregularity.+
    'tbp_lv_perimeterMM',                # Perimeter of lesion (mm).+
    'tbp_lv_radial_color_std_max',       # Color asymmetry, a measure of asymmetry of the spatial distribution of color within the lesion. This score is calculated by looking at the average standard deviation in LAB* color space within concentric rings originating from the lesion center. Values range 0-10.+
    'tbp_lv_stdL',                       # Standard deviation of L inside  lesion.+
    'tbp_lv_stdLExt',                    # Standard deviation of L outside lesion.+
    'tbp_lv_symm_2axis',                 # Border asymmetry; a measure of asymmetry of the lesion's contour about an axis perpendicular to the lesion's most symmetric axis. Lesions with two axes of symmetry will therefore have low scores (more symmetric), while lesions with only one or zero axes of symmetry will have higher scores (less symmetric). This score is calculated by comparing opposite halves of the lesion contour over many degrees of rotation. The angle where the halves are most similar identifies the principal axis of symmetry, while the second axis of symmetry is perpendicular to the principal axis. Border asymmetry is reported as the asymmetry value about this second axis. Values range 0-10.+
    'tbp_lv_symm_2axis_angle',           # Lesion border asymmetry angle.+
    'tbp_lv_x',                          # X-coordinate of the lesion on 3D TBP.+
    'tbp_lv_y',                          # Y-coordinate of the lesion on 3D TBP.+
    'tbp_lv_z',                          # Z-coordinate of the lesion on 3D TBP.+
]

df_train[num_cols] = df_train[num_cols].fillna(df_train[num_cols].median())
df_test [num_cols] = df_test [num_cols].fillna(df_train[num_cols].median())

df_train, new_num_cols, new_cat_cols = feature_engineering(df_train.copy())
df_test, _, _                        = feature_engineering(df_test.copy())

num_cols += new_num_cols

# anatom_site_general
cat_cols = ["sex", "tbp_tile_type", "tbp_lv_location", "tbp_lv_location_simple"] + new_cat_cols
train_cols = num_cols + cat_cols






# df_eff = pd.read_csv("/kaggle/input/isic-inference-effnetv1b0-for-training-data/train_effnetv1b0.csv")
# df_eff = df_eff[["target_effnetv1b0"]]

# df_nex = pd.read_csv("/kaggle/input/nextvit/train_effnetv1b0.csv")
# df_nex = df_nex[["target_effnetv1b0"]]
# df_sel = pd.read_csv("/kaggle/input/selecsls42b-in1k-drop/train_effnetv1b0.csv")
# df_sel = df_sel[["target_effnetv1b0"]]

# df_train["target_nexnetv1b0"] = df_nex["target_effnetv1b0"]
# df_train["target_selnetv1b0"] = df_sel["target_effnetv1b0"]


# df_image_3 = pd.read_csv("/kaggle/input/isic-2024-pl-submission-script-and-preds/train_preds.csv")

# df_train["target_effnetv1b0"] = df_eff["target_effnetv1b0"]
# df_train["target_3"] = df_image_3["pred"]

# train_cols += ["target_3","target_nexnetv1b0","target_effnetv1b0","target_selnetv1b0"]





# ~ + ~ ~ + ~ ~ + ~ ~ + ~ ~ + ~ ~ + ~ ~ + ~ ~ + ~ ~ + ~ ~ + ~
# approximately the same FE does not give anything yet the correct 
# one asks to add one or more parallel lines
# ~ + ~ ~ + ~ ~ + ~ ~ + ~ ~ + ~ ~ + ~ ~ + ~ ~ + ~ ~ + ~ ~ + ~


category_encoder = OrdinalEncoder(
    categories='auto',
    dtype=int,
    handle_unknown='use_encoded_value',
    unknown_value=-2,
    encoded_missing_value=-1,
)

X_cat = category_encoder.fit_transform(df_train[cat_cols])
for c, cat_col in enumerate(cat_cols):
    df_train[cat_col] = X_cat[:, c]

C:\Users\xusha\AppData\Local\Temp\ipykernel_773680\568092388.py:1: DtypeWarning: Columns (51,52) have mixed types. Specify dtype option on import or set low_memory=False.
  df_train = pd.read_csv("../data/train-metadata.csv")


## Cross Validation Setup

In [4]:
N_SPLITS = 5

gkf = StratifiedGroupKFold(n_splits=N_SPLITS, shuffle=True, random_state=42)  

df_train["fold"] = -1

for idx, (train_idx, val_idx) in enumerate(gkf.split(df_train, df_train["target"], groups=df_train["patient_id"])):
    df_train.loc[val_idx, "fold"] = idx

## Competitive Metric

In [5]:
def comp_score(solution: pd.DataFrame, submission: pd.DataFrame, row_id_column_name: str, min_tpr: float=0.80):
    v_gt = abs(np.asarray(solution.values)-1)
    v_pred = np.array([1.0 - x for x in submission.values])
    max_fpr = abs(1-min_tpr)
    partial_auc_scaled = roc_auc_score(v_gt, v_pred, max_fpr=max_fpr)
    # change scale from [0.5, 1.0] to [0.5 * max_fpr**2, max_fpr]
    # https://math.stackexchange.com/questions/914823/shift-numbers-into-a-different-range
    partial_auc = 0.5 * max_fpr**2 + (max_fpr - 0.5 * max_fpr**2) / (1.0 - 0.5) * (partial_auc_scaled - 0.5)
    return partial_auc

def custom_lgbm_metric(y_true, y_hat):
    # TODO: Refactor with the above.
    min_tpr = 0.80
    v_gt = abs(y_true-1)
    v_pred = np.array([1.0 - x for x in y_hat])
    max_fpr = abs(1-min_tpr)
    partial_auc_scaled = roc_auc_score(v_gt, v_pred, max_fpr=max_fpr)
    # change scale from [0.5, 1.0] to [0.5 * max_fpr**2, max_fpr]
    # https://math.stackexchange.com/questions/914823/shift-numbers-into-a-different-range
    partial_auc = 0.5 * max_fpr**2 + (max_fpr - 0.5 * max_fpr**2) / (1.0 - 0.5) * (partial_auc_scaled - 0.5)
    return "pauc80", partial_auc, True

## LGBM Model

With found optimal parameters for model fitting from Kaggle notebook linked earlier ^

In [6]:
new_params = {
    "objective": "binary",
    "verbosity": -1,
    "boosting_type": "gbdt",
    "n_estimators": 200,
    'learning_rate': 0.05,    
    'lambda_l1': 0.0004681884533249742, 
    'lambda_l2': 8.765240856362274, 
    'num_leaves': 136, 
    'feature_fraction': 0.5392005444882538, 
    'bagging_fraction': 0.9577412548866563, 
    'bagging_freq': 6,
    'min_child_samples': 60,
    "device": "gpu"
}
lgb_scores = []
lgb_models = []
oof_df = pd.DataFrame()
for fold in range(N_SPLITS):
    _df_train = df_train[df_train["fold"] != fold].reset_index(drop=True)
    _df_valid = df_train[df_train["fold"] == fold].reset_index(drop=True)
    model = lgb.LGBMClassifier(**new_params)
    # model = VotingClassifier([(f"lgb_{i}", lgb.LGBMClassifier(random_state=i, **new_params)) for i in range(7)], voting="soft")
    model.fit(_df_train[train_cols], _df_train["target"])
    preds = model.predict_proba(_df_valid[train_cols])[:, 1]
    score = comp_score(_df_valid[["target"]], pd.DataFrame(preds, columns=["prediction"]), "")
    print(f"fold: {fold} - Partial AUC Score: {score:.5f}")
    lgb_models.append(model)
    oof_single = _df_valid[["isic_id", "target"]].copy()
    oof_single["pred"] = preds
    oof_df = pd.concat([oof_df, oof_single])

fold: 0 - Partial AUC Score: 0.15745
fold: 1 - Partial AUC Score: 0.16689
fold: 2 - Partial AUC Score: 0.17378
fold: 3 - Partial AUC Score: 0.13681
fold: 4 - Partial AUC Score: 0.15526


In [7]:
lgbm_score = comp_score(oof_df["target"], oof_df["pred"], "")
print(f"LGBM Score: {lgbm_score:.5f}")

LGBM Score: 0.15604


In [8]:
# save model for ensembling later
model.booster_.save_model("./output/lightgbm_model.txt")

## CatBoost Model

With found optimal parameters for model fitting from Kaggle notebook linked earlier ^

In [14]:
cb_params = {
    'objective': 'Logloss',
    # "random_state": 42,
    # "colsample_bylevel": 0.3, # 0.01, 0.1
    "iterations": 400,
    "learning_rate": 0.05,
    "cat_features": cat_cols,
    "max_depth": 8,
    "l2_leaf_reg": 5,
    "task_type": "GPU",
    # "scale_pos_weight": 2,
    "verbose": 0,
}
cb_scores = []
cb_models = []
for fold in range(N_SPLITS):
    _df_train = df_train[df_train["fold"] != fold].reset_index(drop=True)
    _df_valid = df_train[df_train["fold"] == fold].reset_index(drop=True)
    model = cb.CatBoostClassifier(**cb_params)
    # model = VotingClassifier([(f"cb_{i}", cb.CatBoostClassifier(random_state=i, **cb_params)) for i in range(3)], voting="soft")
    # eval_set=(_df_valid[train_cols], _df_valid["target"]), early_stopping_rounds=50
    model.fit(_df_train[train_cols], _df_train["target"])
    preds = model.predict_proba(_df_valid[train_cols])[:, 1]
    score = comp_score(_df_valid[["target"]], pd.DataFrame(preds, columns=["prediction"]), "")
    print(f"fold: {fold} - Partial AUC Score: {score:.5f}")
    cb_scores.append(score)
    cb_models.append(model)

fold: 0 - Partial AUC Score: 0.14568
fold: 1 - Partial AUC Score: 0.15622
fold: 2 - Partial AUC Score: 0.18395
fold: 3 - Partial AUC Score: 0.14203
fold: 4 - Partial AUC Score: 0.15104


In [15]:
cb_score = np.mean(cb_scores)
print(f"CatBoost Score: {cb_score:.5f}")

CatBoost Score: 0.15579


In [17]:
# save model for ensembling later
model.save_model("./output/catboost_model.cbm")